In [3]:

import os
from PIL import Image
import numpy as np
import pickle
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.neighbors import NearestNeighbors
from numpy.linalg import norm

In [4]:
feature_list = np.array(pickle.load(open('embeddings.pkl','rb')))
filenames = pickle.load(open('filenames.pkl','rb'))

In [8]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False



In [9]:
model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

In [11]:
img = image.load_img('sample/1541.jpg',target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()
normalized_result = result / norm(result)


In [12]:
neighbors = NearestNeighbors(n_neighbors=6,algorithm='brute',metric='euclidean')
neighbors.fit(feature_list)


NearestNeighbors(algorithm='brute', metric='euclidean', n_neighbors=6)

In [13]:
distances,indices = neighbors.kneighbors([normalized_result])

In [14]:
print(indices)

[[ 4565  4574 18322 23930 25830   105]]


In [16]:
for file in indices[0]:
    print(filenames[file])

images\1541.jpg
images\1542.jpg
images\3177.jpg
images\38675.jpg
images\4121.jpg
images\10125.jpg


In [18]:
import cv2

In [19]:
for file in indices[0][1:6]:
    temp_img = cv2.imread(filenames[file])
    cv2.imshow('output',cv2.resize(temp_img,(512,512)))
    cv2.waitKey(0)